In [1]:
from sccomp_glm_data_frame_counts import sccomp_glm_data_frame_counts
import pandas as pd

In [2]:
count_obj = pd.read_csv('./data/count_obj.csv')
count_obj

,sample,type,phenotype,count,cell_group,proportion
0,10x_6K,benign,b_cell_macrophage_precursor_or_follicular_LTB_...,42,BM,0.008350
1,10x_6K,benign,B_cell:immature,361,B1,0.071769
2,10x_6K,benign,B_cell:immature_IGLC3_IGLC2,57,B2,0.011332
3,10x_6K,benign,B_cell:Memory_ITM2C_IGHA1_MZB1_JCHAIN,40,B3,0.007952
4,10x_6K,benign,Dendritic_CD11_CD1_high_mito,75,Dm,0.014911
...,...,...,...,...,...,...
715,SRR7244582,benign,T_cell:CD8+_GZMK_DUSP2_LYAR_CCL5,197,CD8 2,0.060727
716,SRR7244582,benign,T_cell:CD8+_non_activated,320,CD8 3,0.098644
717,SRR7244582,benign,T_cell:CD8+_PPBP_SAT1,39,CD8 4,0.012022
718,SRR7244582,benign,T_cell:CD8+_S100B,88,CD8 5,0.027127


In [3]:
res = sccomp_glm_data_frame_counts(
    data = count_obj,
    formula_composition = '~ 0 + type', 
    sample = 'sample',
    cell_group = 'cell_group',
    count = 'count'
)

11:54:00 - cmdstanpy - INFO - Chain [1] start processing
11:54:00 - cmdstanpy - INFO - Chain [2] start processing
11:54:00 - cmdstanpy - INFO - Chain [3] start processing
11:54:00 - cmdstanpy - INFO - Chain [4] start processing


Chain [1] method = sample (Default)Chain [2] method = sample (Default)
Chain [2] sample
Chain [2] num_samples = 1000 (Default)
Chain [2] num_warmup = 1000 (Default)
Chain [2] save_warmup = false (Default)
Chain [2] thin = 1 (Default)
Chain [2] adapt
Chain [2] engaged = true (Default)
Chain [2] gamma = 0.05 (Default)
Chain [2] delta = 0.8 (Default)
Chain [2] kappa = 0.75 (Default)
Chain [2] t0 = 10 (Default)
Chain [2] init_buffer = 75 (Default)
Chain [2] term_buffer = 50 (Default)
Chain [2] window = 25 (Default)
Chain [2] save_metric = false (Default)
Chain [2] algorithm = hmc (Default)
Chain [2] hmc
Chain [2] engine = nuts (Default)
Chain [2] nuts
Chain [2] max_depth = 10 (Default)
Chain [2] metric = diag_e (Default)
Chain [2] metric_file =  (Default)
Chain [2] stepsize = 1 (Default)
Chain [2] stepsize_jitter = 0 (Default)
Chain [2] num_chains = 1 (Default)
Chain [2] id = 2
Chain [2] data
Chain [2] file = /tmp/tmphc97wmx5/o1irbj8c.json
Chain [2] init = 2 (Default)
Chain [2] random
Chai

Chain [4] Iteration:  100 / 2000 [  5%]  (Warmup)
Chain [1] Iteration:  100 / 2000 [  5%]  (Warmup)
Chain [2] Iteration:  100 / 2000 [  5%]  (Warmup)
Chain [3] Iteration:  100 / 2000 [  5%]  (Warmup)
Chain [1] Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain [4] Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain [2] Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain [3] Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain [1] Iteration:  300 / 2000 [ 15%]  (Warmup)
Chain [4] Iteration:  300 / 2000 [ 15%]  (Warmup)
Chain [2] Iteration:  300 / 2000 [ 15%]  (Warmup)
Chain [3] Iteration:  300 / 2000 [ 15%]  (Warmup)
Chain [1] Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain [4] Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain [2] Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain [3] Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain [1] Iteration:  500 / 2000 [ 25%]  (Warmup)
Chain [4] Iteration:  500 / 2000 [ 25%]  (Warmup)
Chain [2] Iteration:  500 / 2000 [ 25%]  (Warmup)
Chain [3] Iteration:  500 / 2000 [ 25%]  (Warmup)


11:54:08 - cmdstanpy - INFO - Chain [1] done processing
11:54:09 - cmdstanpy - INFO - Chain [2] done processing
11:54:09 - cmdstanpy - INFO - Chain [4] done processing
11:54:09 - cmdstanpy - INFO - Chain [3] done processing
11:54:09 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: gamma_lpdf: Random variable is inf, but must be positive finite! (in 'glm_multi_beta_binomial.stan', line 753, column 2 to column 53)
	Exception: gamma_lpdf: Random variable is inf, but must be positive finite! (in 'glm_multi_beta_binomial.stan', line 753, column 2 to column 53)
	Exception: Exception: beta_binomial_lpmf: First prior sample size parameter[2] is inf, but must be positive finite! (in 'glm_multi_beta_binomial.stan', line 214, column 16 to line 219, column 19) (in 'glm_multi_beta_binomial.stan', line 653, column 3 to line 683, column 8)
	Exception: gamma_lpdf: Random variable is inf, but must be positive finite! (in 'glm_multi_beta_binomial.stan', line 753, column 2 to column 53

Chain [1] Iteration: 2000 / 2000 [100%]  (Sampling)
Chain [1] 
Chain [1] Elapsed Time: 4.488 seconds (Warm-up)
Chain [1] 4.001 seconds (Sampling)
Chain [1] 8.489 seconds (Total)
Chain [1] 
Chain [1] 
Chain [2] Iteration: 2000 / 2000 [100%]  (Sampling)
Chain [4] Iteration: 2000 / 2000 [100%]  (Sampling)
Chain [2] 
Chain [2] Elapsed Time: 4.554 seconds (Warm-up)
Chain [2] 4.001 seconds (Sampling)
Chain [2] 8.555 seconds (Total)
Chain [2] 
Chain [2] 
Chain [4] 
Chain [4] Elapsed Time: 4.538 seconds (Warm-up)
Chain [4] 4.008 seconds (Sampling)
Chain [4] 8.546 seconds (Total)
Chain [4] 
Chain [4] 
Chain [3] Iteration: 2000 / 2000 [100%]  (Sampling)
Chain [3] 
Chain [3] Elapsed Time: 4.592 seconds (Warm-up)
Chain [3] 4.002 seconds (Sampling)
Chain [3] 8.594 seconds (Total)
Chain [3] 
Chain [3] 


In [4]:
res

{'fit': CmdStanMCMC: model=glm_multi_beta_binomial chains=4['method=sample', 'algorithm=hmc', 'adapt', 'engaged=1']
  csv_files:
 	/tmp/tmphc97wmx5/glm_multi_beta_binomialbiv45b3t/glm_multi_beta_binomial-20241121115400_1.csv
 	/tmp/tmphc97wmx5/glm_multi_beta_binomialbiv45b3t/glm_multi_beta_binomial-20241121115400_2.csv
 	/tmp/tmphc97wmx5/glm_multi_beta_binomialbiv45b3t/glm_multi_beta_binomial-20241121115400_3.csv
 	/tmp/tmphc97wmx5/glm_multi_beta_binomialbiv45b3t/glm_multi_beta_binomial-20241121115400_4.csv
  output_files:
 	/tmp/tmphc97wmx5/glm_multi_beta_binomialbiv45b3t/glm_multi_beta_binomial-20241121115400_0-stdout.txt
 	/tmp/tmphc97wmx5/glm_multi_beta_binomialbiv45b3t/glm_multi_beta_binomial-20241121115400_1-stdout.txt
 	/tmp/tmphc97wmx5/glm_multi_beta_binomialbiv45b3t/glm_multi_beta_binomial-20241121115400_2-stdout.txt
 	/tmp/tmphc97wmx5/glm_multi_beta_binomialbiv45b3t/glm_multi_beta_binomial-20241121115400_3-stdout.txt,
 'model_input': {'N': 20,
  'M': 36,
  'exposure': array([

In [5]:
res.get('fit').summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
lp__,-478862.000000,0.269348,7.681770,-478875.000000,-478862.000000,-478850.000000,813.381,50.7982,1.006540
"beta_raw_raw[1,1]",-0.849857,0.001315,0.091429,-0.996311,-0.852285,-0.695751,4832.020,301.7750,1.000260
"beta_raw_raw[1,2]",-0.560615,0.001678,0.115965,-0.745049,-0.562825,-0.369626,4775.290,298.2320,0.999332
"beta_raw_raw[1,3]",0.223466,0.001409,0.103632,0.051842,0.224624,0.394587,5411.700,337.9780,0.999747
"beta_raw_raw[1,4]",0.695064,0.001708,0.106722,0.521281,0.695455,0.871706,3903.260,243.7710,0.999879
...,...,...,...,...,...,...,...,...,...
log_lik[716],0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
log_lik[717],0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
log_lik[718],0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
log_lik[719],0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN


In [6]:
from utilities import draws_to_tibble_x_y
draws_to_tibble_x_y(res.get('fit'), 'beta', 'C', 'M')

,chain__,iter__,draw__,variable,C,M,value
0,1.0,1.0,1,beta,1,1,1.143120
1,1.0,2.0,2,beta,1,1,1.023800
2,1.0,3.0,3,beta,1,1,1.340230
3,1.0,4.0,4,beta,1,1,0.993754
4,1.0,5.0,5,beta,1,1,1.245330
...,...,...,...,...,...,...,...
287995,4.0,996.0,3996,beta,2,36,-2.718980
287996,4.0,997.0,3997,beta,2,36,-2.500240
287997,4.0,998.0,3998,beta,2,36,-2.206970
287998,4.0,999.0,3999,beta,2,36,-2.303860


The following is for dev:

In [7]:
data = res
# Retrieve attributes from data
cell_group = data.get("cell_group", None)
model_input = data.get("model_input", {})
fit = data.get("fit", {})

In [8]:
beta_factor_of_interest = data.get('model_input').get("X").columns.tolist()
beta = draws_to_tibble_x_y(fit, "beta", "C", "M")
beta = beta.pivot(index = ['chain__', 'iter__', 'draw__', 'variable', 'M'], columns='C', values='value')
beta.columns = beta_factor_of_interest
beta.reset_index(inplace=True)

# Abundance
draws = beta.drop(columns=['variable'], errors='ignore')

In [9]:
contrasts = ['type[cancer] - type[benign]']

In [10]:
from utilities import mutate_from_expr_list
mutate_from_expr_list(x = draws, formula_expr = contrasts )

,chain__,iter__,draw__,M,type[benign],type[cancer],type[cancer] - type[benign]
0,1.0,1.0,1,1,1.143120,0.477964,-0.665156
1,1.0,1.0,1,2,0.830498,0.056474,-0.774024
2,1.0,1.0,1,3,-0.328884,-0.637806,-0.308922
3,1.0,1.0,1,4,-0.774410,-1.461950,-0.687540
4,1.0,1.0,1,5,0.357966,0.558934,0.200968
...,...,...,...,...,...,...,...
143995,4.0,1000.0,4000,32,0.649124,0.574817,-0.074307
143996,4.0,1000.0,4000,33,0.751803,0.886668,0.134865
143997,4.0,1000.0,4000,34,0.102439,0.455362,0.352923
143998,4.0,1000.0,4000,35,-0.842448,-0.840790,0.001658


In [11]:
from utilities import summary_to_tibble
summary_to_tibble(fit, "alpha_normalised", "C", "M")

,variable,C,M,Mean,5%,25%,50%,75%,95%,N_Eff,R_hat
0,alpha_normalised,1,1,5.38144,4.82949,5.16585,5.38912,5.60650,5.90650,5611.46,0.999405
1,alpha_normalised,1,2,5.03838,4.53155,4.83800,5.04452,5.24358,5.53358,6005.05,0.999986
2,alpha_normalised,1,3,5.85659,5.27205,5.62683,5.85791,6.09527,6.43844,6138.21,0.999293
3,alpha_normalised,1,4,6.21188,5.57909,5.96114,6.20855,6.46829,6.83667,6440.13,0.999612
4,alpha_normalised,1,5,6.29355,5.66902,6.03759,6.29752,6.55653,6.90581,5667.67,0.999909
5,alpha_normalised,1,6,5.13442,4.63103,4.92416,5.13831,5.34823,5.62555,5119.00,1.000020
6,alpha_normalised,1,7,4.71193,4.17792,4.49417,4.71409,4.93979,5.23380,5807.25,0.999345
7,alpha_normalised,1,8,5.93207,5.19433,5.62024,5.92360,6.24274,6.69850,4331.13,1.000230
8,alpha_normalised,1,9,5.24804,4.74017,5.05056,5.25075,5.45764,5.73204,5960.00,0.999626
9,alpha_normalised,1,10,5.46210,4.85896,5.22746,5.47126,5.70202,6.03810,5634.35,1.000030


In [12]:
from utilities import get_abundance_contrast_draws
get_abundance_contrast_draws(data, contrasts)

,cell_group,M,chain__,iter__,draw__,parameter,value,N_Eff,R_k_hat
0,B1,1,1.0,1.0,1,type[benign],1.143120,4832.02,1.00026
285339,M6,26,3.0,667.0,2667,type[benign],-0.822926,6027.51,1.00007
285338,M6,26,3.0,667.0,2667,type[benign],-0.822926,6027.51,1.00007
285337,M6,26,3.0,667.0,2667,type[benign],-0.822926,5007.94,1.00059
285336,M5,25,3.0,667.0,2667,type[benign],-0.089573,1765.32,1.00097
...,...,...,...,...,...,...,...,...,...
904000,CD8 4,13,2.0,334.0,1334,type[cancer] - type[benign],-0.440585,NaN,NaN
904001,CD8 5,14,2.0,334.0,1334,type[cancer] - type[benign],-0.469055,NaN,NaN
904002,CD8 6,15,2.0,334.0,1334,type[cancer] - type[benign],0.094142,NaN,NaN
903996,CD4 5,9,2.0,334.0,1334,type[cancer] - type[benign],0.412714,NaN,NaN


In [13]:
# Extract attributes
sample = data.get("sample")
cell_group = data.get("cell_group")
count = data.get("count")
model_input = data.get("model_input")
truncation_df2 = data.get("truncation_df2")

In [14]:
# Abundance
abundance_CI = get_abundance_contrast_draws(data, contrasts)

In [15]:
draws = abundance_CI
percent_false_positive = 5 
test_composition_above_logit_fold_change=0.1

In [16]:
from utilities import draws_to_statistics
draws_to_statistics(abundance_CI, percent_false_positive / 100,
            test_composition_above_logit_fold_change,
            cell_group,
            "c_")

,cell_group,M,parameter,c_lower,c_effect,c_upper,c_pH0,FDR,N_Eff,R_k_hat
0,B1,1,type[benign],0.895225,1.158380,1.394426,0.00000,0.000000,4832.020,1.000260
1,B1,1,type[cancer],0.148726,0.498919,0.825807,0.01375,0.001390,4254.190,0.999630
2,B1,1,type[cancer] - type[benign],-1.070146,-0.659320,-0.251411,0.00450,0.001100,NaN,NaN
3,B2,2,type[benign],0.408081,0.732392,1.035481,0.00125,0.000095,4820.020,0.999329
4,B2,2,type[cancer],-0.406545,0.032374,0.411927,0.64125,0.087949,5398.290,0.999668
...,...,...,...,...,...,...,...,...,...,...
103,TM2,35,type[cancer],-1.213822,-0.894583,-0.572024,0.00000,0.000000,4236.530,1.000480
104,TM2,35,type[cancer] - type[benign],-0.159803,0.279882,0.717904,0.20825,0.054917,NaN,NaN
105,TM3,36,type[benign],-1.765449,-0.873766,0.162452,0.07175,0.009050,994.182,1.003410
106,TM3,36,type[cancer],-4.033643,-2.751570,-1.532792,0.00000,0.000000,724.281,1.000360


In [17]:
from utilities import get_variability_contrast_draws
get_variability_contrast_draws(data, None)

,M,chain__,iter__,draw__,cell_group,parameter,value,N_Eff,R_k_hat
0,1,1.0,1.0,1,B1,Intercept,-4.83133,5611.46,0.999405
1,1,1.0,2.0,2,B1,Intercept,-5.12614,5611.46,0.999405
2,1,1.0,3.0,3,B1,Intercept,-5.64359,5611.46,0.999405
3,1,1.0,4.0,4,B1,Intercept,-5.31339,5611.46,0.999405
4,1,1.0,5.0,5,B1,Intercept,-5.42776,5611.46,0.999405
...,...,...,...,...,...,...,...,...,...
143995,36,4.0,996.0,3996,TM3,Intercept,-2.98418,1088.21,1.000650
143996,36,4.0,997.0,3997,TM3,Intercept,-3.01434,1088.21,1.000650
143997,36,4.0,998.0,3998,TM3,Intercept,-2.88513,1088.21,1.000650
143998,36,4.0,999.0,3999,TM3,Intercept,-3.44640,1088.21,1.000650
